In [2]:
'''
****************************
* consolidation of protein scoring routines into a single module.
* 5 Sep 2018
* separated out into a module to enable independent tesign and version control
* 14 Sep 2018 include randomising by mask
*
* 22 Sep 2018
* added randomseed to randomising routines
* allows seed to be set externally and hence allows reproducibility where required.
*
* 8 Nov 2018
* change the scoring of X as requested by Tim
* currently "X" scores
* 2
* 4 2
* 2 2
* 2 2    across the 7 main residues. This should be changed to

* 2
* 4 2
* 2 2
* 0 2 
* 
* and also add 'XXXXXX' on the front and 'XXXX' on the end
* this done via an argument
*
* 13 Nov 2018
* added method score_acidic_tract for just the specific acid tract prior to the FFAT motif.
*
* 23 May 2019
* adding a new class for a separate scoring model - mail from Tim 21 May 2019
* for Fabien Alpy
* "would like to see what the genome has for motifs with S/T at position 4 and no prejudice for S/T at all other positions."
*
* 22 April 2020
* forced upgrade to python 3
* print 'xxx' replaced with print ('xxx')
* changes to lists and ranges showing up in random.shuddle) so replacing range(0,n) with list(range(0,n))
* 
****************************
'''

import random # randomising sequences

class SequenceAnalysis():
    
    def __init__(self):
        '''
        ****************************
        * 4 Aug 2018
        * change of weightings from previous version
        * inclusion of * as an amino acid with its own set of weightings
        * 10 Aug inclusion of U same weightings as K for human proteins
        * 
        ****************************
        '''
    
        self.versionid = 'Dev on 1.0'
    
        self.aminoacids = 'ACDEFGHIKLMNPQRSTUVWXY'
        self.aminoacids += 'Z'# special because Z crops up in one proteome
        #self.aminoacids += '*'# occurs in some proteomes so added here - 22 Sep 18

        self.aminoscores= []
        self.aminoscores.append(['DE',0])
        self.aminoscores.append(['ST',0.5])
        self.aminoscores.append(['ACFGHILMNPQVWY',1])
        self.aminoscores.append(['KRUXZ*',2])

        self.aminoacidgroups = []
        self.aminoacidgroups.append([1,'DE'])
        self.aminoacidgroups.append([2,'ST'])
        self.aminoacidgroups.append([3,'FYW'])
        self.aminoacidgroups.append([4,'UKRHXZ'])
        self.aminoacidgroups.append([5,'ACG'])
        self.aminoacidgroups.append([6,'ILMV'])
        self.aminoacidgroups.append([7,'NQ'])
        self.aminoacidgroups.append([8,'P'])

        self.flankrule = [[0,0],[2.0,0.5],[3.0,1.0],[4.0,1.5]]

        self.weightings = []

        self.weightings.append(['*',[2,4,2,2,2,2,2]])
        self.weightings.append(['A',[1,4,1,2,0,0,1]])
        self.weightings.append(['C',[1,4,1,2,0,0,1]])
        self.weightings.append(['D',[0,4,1,0,4,0,0]])
        self.weightings.append(['E',[0,4,1,0,4,0,0]])
        self.weightings.append(['F',[1,0,0,2,2,0,1]])
        self.weightings.append(['G',[1,4,1,2,1,0,1]])
        self.weightings.append(['H',[1,4,0.7,2,2,0,1.5]])
        self.weightings.append(['I',[1,4,1,2,2,0,1]])
        self.weightings.append(['K',[1.5,4,1,2,2.5,0,1.5]])
        self.weightings.append(['L',[1,4,1,2,2,0,1]])
        self.weightings.append(['M',[1,4,1,2,2,0,1]])
        self.weightings.append(['N',[1,4,1,2,2,0,1]])
        self.weightings.append(['P',[1,4,1,2,2,0,1]])
        self.weightings.append(['Q',[1,4,1,2,2,0,1]])
        self.weightings.append(['R',[2,4,1,2,3,0,1.5]])
        self.weightings.append(['S',[0.5,4,1,0.5,0.5,0,0.5]])
        self.weightings.append(['T',[0.5,4,1,0.5,0.5,0,0.5]])
        self.weightings.append(['U',[1.5,4,1,2,2.5,0,1.5]])#10 Aug for human protein
        self.weightings.append(['V',[1,4,1,2,2,0,1]])
        self.weightings.append(['W',[1,2,0.7,2,2,0,1]])
        self.weightings.append(['Y',[1,0, 0.5,2,2,0,1]])
        self.weightings.append(['X',[2,4,2,2,2,0,2]])
        self.weightings.append(['Z',[2,4,2,2,2,2,2]])
        
    def score_acidic_tract(self, sequence):
        
        '''
        ***********************************
        * 13 Nov 2018
        * scores the acidic tract separately
        * currently does not check length of sequence
        ***********************************
        '''
        
        counttract = 0
        myscore = 99
        
        for i in range(0,6):
            for score in self.aminoscores:
                if sequence[i] in score[0]:
                    counttract += score[1]
                    
        for flankscore in self.flankrule:
            if counttract > flankscore[0]:
                myscore = flankscore[1]
               
        return myscore

    def score_sequence(self, sequence):
        
        '''
        ***********************************
        * applies the FFAT weighting algorithm to a sequence.
        ***********************************
        '''
        myscore = 0

        if len(sequence) < 13:
            return -1
        
        countflank = 0.0
        for i in range(0, 6):
            for score in self.aminoscores:
                if sequence[i] in score[0]:
                    countflank += score[1]
        
        for flankscore in self.flankrule:
            if countflank > flankscore[0]:
                myscore = flankscore[1]

        for i in range(6,13):
            
            idx = [x[0] for x in self.weightings].index(sequence[i])
            weight = self.weightings[idx]
            myscore += weight[1][i - 6]
        
        return myscore
    
    def score_protein(self, protein, bextended = True):
        
        '''
        ***********************************
        * returns the scores for a protein.
        * currently does not add a string at the beginning to allow the first aminoacid string to be scored in the weightings
        * nor pad out the last
        *
        * 8 Nov 2018 - pad out both ends with 'X' 6 at the beginning and 4 at the end.
        * 
        ***********************************
        '''
        
        if bextended:
            myextendedprotein = 'XXXXXX' + protein[:] + 'XXXX'
        else:
            myextendedprotein = protein[:]
    
        nump = len(myextendedprotein)
        myscores = []
        for i in range(0,nump-14):
            myscores.append(self.score_sequence(myextendedprotein[i : i + 13]))
        
        return myscores
    
    def score_protein_acidic_tract(self, protein):
        
        nump = len(protein)
        myscores = []
        for i in range(0, nump - 7):
            myscores.append(self.score_acidic_tract(protein[i : i + 6]))
        
        return myscores
    
    def protein_score_ordered(self, rawscores, limit = 0):
        
        '''
        ***********************************
        * takes in the raw output of score_protein
        * returns the scores sequenced from the lowest score to the highest
        * score_protein returns a list of scores, eg [1.5, 3.4, 2.5, 0.5]
        * this returns pairs, ie
        * [[0.5, 3], [1.5, 0], [2.5, 2], [3.4, 1]]
        *
        * 17 Sep 2018
        * sorting a 3000 length sequence when we just want the minimum 10 is a 
        * significant waste of CPU time so introduce an optional limit
        * set to zero as default which means do the whole lot 
        ***********************************
        '''
        
        lenN = len(rawscores)
        seq = list(range(0,lenN))
        
        if limit > 0:
            countlimit = min( limit + 1, lenN - 1)
        else:
            countlimit = lenN - 1
            
        for i in range (0, countlimit):
            
            for j in range(lenN - 2, i-1, -1):
                
                if rawscores[seq[j]] > rawscores[seq[j+1]]:
                    temp = seq[j]
                    seq[j] = seq[j+1]
                    seq[j+1] = temp
        
        myorderedscores = []
        
        for i in range (0,lenN):
            myorderedscores.append([rawscores[seq[i]], seq[i]])
            
        for score in myorderedscores:
            if score[1] < 0:
                print (score)
            
        return myorderedscores
    
    
    def randomise_proteins_masked(self, proteins, mask, randomseed, bbyprotein = False):
        '''
        ***********************************
        * 1 Sep 2018
        * takes a list of proteins in and produces a randomised version
        * stores the numbers of each amino acids
        * produces a sequence of proteins of the same number and lengths as the incoming file
        * and same number of amino acids in a random sequence
        * 10 Sep 2018
        * masked version. 
        * the mask is a string of 0's and 1's. Metaphorically the proteins are like a giant scrabble board
        * a 1 in the mask means we keep that tile in its place. 0 means we pick it up
        * put it in the bag, mix it up with the others, and put it down in a random place.
        * proteome is [[file details],[proteins]]
        * and protines are [[protein 1 name and details],[protein code]]
        * 
        * 22 Sep 2018
        * random seed added. Passed in externally
        * change in randomisation to ensure same count of amino acids
        ***********************************
        '''
        
        random.seed(randomseed)
        
        numacids = len(self.aminoacids)   
        
        if bbyprotein:
            
            myrandomproteome = []
            
            numproteins = len(proteins)
            for i in range(0, numproteins):
                
                lenprotein = len(proteins[i][1])
                ctracids = lenprotein - sum(mask[i])
                
                # create  string of the amino acids in this protein
                # only consider those that are to be shuffled i.e. mask == 0

                acidstomix = ''
            
                for j in range(0, lenprotein):
                    if mask[i][j] == 0:
                        acidstomix += proteins[i][1][j]

                # create massive string of the amino acids in same number

                selectionsequence = ''
                for acid in self.aminoacids:
                    acidcount = acidstomix.count(acid) 
                    selectionsequence += acid * acidcount

                #some proteins can contain acids not in our list.

                nonacids = ''
                for aa in acidstomix:
                    if not aa in self.aminoacids:
                        nonacids += aa

                if len(nonacids)>0:
                    print ('the following non-acids found: ' + nonacids)
                    selectionsequence += nonacids


                # create a randomised index aaindex which we will use sequentially.
                # using random.shuffle!
                # and use ctrindex to plough through

                numchain = len(selectionsequence)
                aaindex = list(range(0, numchain))
                random.shuffle(aaindex)
                ctrindex = 0

                # programmer inefficiency here
                # numchain, ctracids should be the same

                newID = proteins[i][0][0] + ' - randomised'
                thischain = ''
                ctrstatic = 0

                for j in range(0, lenprotein):

                    if mask[i][j] == 0:

                        thischain += selectionsequence[aaindex[ctrindex]]
                        ctrindex += 1

                    else:

                        ctrstatic +=1
                        thischain += proteins[i][1][j]

                myrandomproteinheader = [newID,'static amino acids - ' + str(ctrstatic)]
                myrandomprotein = [myrandomproteinheader, thischain]
                myrandomproteome.append(myrandomprotein)
            
            
        else:
            
             
            aacount = []

            ctracids = 0
            numproteins = len(proteins)
            for i in range(0, numproteins):
                ctracids += len(proteins[i][1])
                ctracids -= sum(mask[i]) # uses the fact that the mask is just 1's and 0's

            # create massive string of the amino acids in the proteome
            # only consider those that are to be shuffled i.e. mask == 0

            acidstomix = ''
            for i in range(0, numproteins):
                numacidsinchain = len(proteins[i][1])
                for j in range(0, numacidsinchain):
                    if mask[i][j] == 0:
                        acidstomix += proteins[i][1][j]

            # create massive string of the amino acids in same number

            selectionsequence = ''
            for acid in self.aminoacids:
                acidcount = acidstomix.count(acid) 
                selectionsequence += acid * acidcount

            #some proteins can contain acids not in our list.

            nonacids = ''
            for aa in acidstomix:
                if not aa in self.aminoacids:
                    nonacids += aa

            if len(nonacids)>0:
                print ('the following non-acids found: ' + nonacids)
                selectionsequence += nonacids


            # create a randomised index aaindex which we will use sequentially.
            # using random.shuffle!
            # and use ctrindex to plough through

            numchain = len(selectionsequence)
            aaindex = list(range(0, numchain))
            random.shuffle(aaindex)
            ctrindex = 0

            # programmer inefficiency here
            # numchain, ctracids should be the same

            myrandomproteome = []

            for iprotein in range(0, numproteins):

                lenthisprotein = len(proteins[iprotein][1])
                newID = proteins[iprotein][0][0] + ' - randomised'
                thischain = ''
                ctrstatic = 0

                for j in range(0, lenthisprotein):

                    if mask[iprotein][j] == 0:

                        thischain += selectionsequence[aaindex[ctrindex]]
                        ctrindex += 1

                    else:

                        ctrstatic +=1
                        thischain += proteins[iprotein][1][j]

                myrandomproteinheader = [newID,'static amino acids - ' + str(ctrstatic)]
                myrandomprotein = [myrandomproteinheader, thischain]
                myrandomproteome.append(myrandomprotein)
                            
        return myrandomproteome
    
    def randomise_proteins(self, proteins, randomseed, bbyprotein = False):
        '''
        ***********************************
        * 1 Sep 2018
        * takes a proteome in and produces a randomised version
        * stores the numbers of each amino acids
        * produces a sequence of proteins of the same number and lengths as the incoming file
        * and same number of amino acids in a random sequence
        * 5 Sep 2018 modified to takew the full proteome structure.
        * 14 Sep changed to use the masked version
        ***********************************
        '''
        
        blankmask = []

        for protein in proteins:

            numacids = len(protein[1])
            proteinmask = [0] * numacids

            blankmask.append(proteinmask)
        
        return self.randomise_proteins_masked(proteins,blankmask, randomseed, bbyprotein)
    
    def generate_mask_preserve_acidic_flank(self, proteins, maxscore = 0.5, bbyprotein = False):
        '''
        ***********************************
        * 13 Dec 2018
        * just produces a mask
        * takes a proteome and produces a randomised version
        * preserves acidic aminoacids 'DEST in the flank, but
        * only with a sequence scoring 0.5 or less
        ***********************************
        '''
        
        blankmask = []
        ctracidsintract = 0
        qualifyingacids = 'DEST'

        for protein in proteins:

            numacids = len(protein[1])
            proteinmask = [0] * numacids
            
            myresults = self.score_protein_acidic_tract(protein[1])
            for i in range(0, len(myresults)):
                if myresults[i] <= maxscore:
                    # eureka - have found a qualifying segment.
                    # check the flank for scidic amino acids
                    # flank is 0-6 of the segment
                    for j in range(0, 6):
                        if protein[1][i + j] in qualifyingacids:
                            proteinmask[i + j] = 1
                            ctracidsintract += 1
                    
            blankmask.append(proteinmask)
            
        return blankmask

    def randomise_proteins_preserve_acidic_flank(self, proteins, randomseed, maxscore = 0.5, bbyprotein = False):
        '''
        ***********************************
        * 13 Dec 2018
        * refactored
        ***********************************
        '''

        mask = self.generate_mask_preserve_acidic_flank(proteins, maxscore, bbyprotein)
        
        return self.randomise_proteins_masked(proteins, mask, randomseed, bbyprotein)
    
    def aminoacid_count(self, proteins):
        
        '''
        ***********************************
        * 16 Sep 2018
        * a utility for counting the number of amino acids in a proteome
        * a check when randomising, and who knows what else
        ***********************************
        '''
        numacids = len(self.aminoacids)
        aacount = [0] * numacids
        
        for protein in proteins:
            for i in range(0,numacids):
                aacount[i] += protein[1].count(self.aminoacids[i]) 
         
        return [self.aminoacids, aacount]
        
    def score_sequence_reordered(self, sequence, reorder):
        
        '''
        ***********************************
        * 28 Aug 2018
        * a variant of scoresequence
        * a re-order is passed in, eg 3672154
        * which has to be passed in as 2561043
        * it needs to have all the integers between 0 and 6 once and once only.
        * I do not check for this.
        *
        * the reorder applies to weightings
        *  weightings.append(['*',[2,4,2,2,2,2,2]])
        * so for amino acids in range (6-13), instead of multiplying by weight[1][i-6]
        * we multiply by weight[1][reorder[i - 6]]
        * where weight = self.weightings row
        ***********************************
        '''
        
        myscore = 0
        #mytally = [sequence,[]]

        if len(sequence)<13:
            return -1
        
        countflank = 0.0
        for i in range(0, 6):
            for score in self.aminoscores:
                if sequence[i] in score[0]:
                    countflank += score[1]
                    #mytally[1].append(score[1])
        
        for flankscore in self.flankrule:
            if countflank > flankscore[0]:
                myscore = flankscore[1]

        for i in range(6,13):
            
            idx = [x[0] for x in self.weightings].index(sequence[i])
            weight = self.weightings[idx]
            myscore += weight[1][reorder[i - 6]]
            #mytally[1].append([sequence[i],weight[1][i - 6]])
        
        return myscore
    
    
    def score_protein_reordered(self, protein, reorder, bextended = True):
    
        '''
        ***********************************
        * 8 Nov 2018 - pad out both ends with 'X' 6 at the beginning and 4 at the end.
        * 
        ***********************************
        '''
        if bextended:
            myextendedprotein = 'XXXXXX' + protein[:] + 'XXXX'
        else:
            myextendedprotein = protein[:]
            
        nump = len(myextendedprotein)
        myscores = []
        for i in range(0,nump-14):
            myscores.append(self.score_sequence_reordered(myextendedprotein[i : i + 13],reorder))
        
        return myscores
    
    def acidity_score_protein(self, protein, stublength = 6):
        
        '''
        ***********************************
        * an idea from Tim.
        * measure the amount of acidity in a short section and return in historgram form.
        * ED = 1
        * ST 0.5
        * KR -H -0.5
        * Others 0
        ***********************************
        '''
        myscore = []
        acidscore = [['ED',1.0],['ST',0.5],['KRH',-0.5]]
        
        lenprotein = len(protein)
        bhavescore = False
        currentscore = 0
        
        for i in range(0, (1 + lenprotein - stublength)):
            if not bhavescore:
                for j in range(i, i + stublength):
                    for acid in acidscore:
                        if protein[j] in acid[0]:
                            currentscore += acid[1]
                myscore.append(currentscore)
                bhavescore = True
            else:
                for acid in acidscore:
                    if protein[i - 1] in acid[0]:
                        currentscore -= acid[1]
                    if protein[i + stublength - 1] in acid[0]:
                        currentscore += acid[1]
                myscore.append(currentscore)
        
        return myscore
    
 
'''
****************************
* end of the scorer class
****************************
'''
print ('Scorer Dev GH 22 01')

Scorer Dev GH 22 01
